In [1]:
# refacto V3 :
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import csv

ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}

def extraire_donnees_v3(fichier_xml: str) -> pd.DataFrame:
    try:
        tree = ET.parse(fichier_xml)
        root = tree.getroot()
    except Exception as e:
        print(f"❌ Erreur de parsing {fichier_xml} : {e}")
        return pd.DataFrame()

    def get_text(path):
        elem = root.find(path, ns)
        return elem.text.strip() if elem is not None and elem.text else ""

    # Métadonnées
    metadata = {
        "UID": get_text(".//ns:uid"),
        "SeanceRef": get_text(".//ns:seanceRef"),
        "SessionRef": get_text(".//ns:sessionRef"),
        "DateSeance": get_text(".//ns:dateSeance"),
        "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
        "NumSeanceJour": get_text(".//ns:numSeanceJour"),
        "NumSeance": get_text(".//ns:numSeance"),
        "TypeAssemblee": get_text(".//ns:typeAssemblee"),
        "Legislature": get_text(".//ns:legislature"),
        "Session": get_text(".//ns:session"),
        "NomFichierJO": get_text(".//ns:nomFichierJo"),
        "President": get_text(".//ns:presidentSeance"),
    }

    rows = []
    titre_general = ""
    sous_titre = ""

    contenu = root.find("ns:contenu", ns)
    if contenu is None:
        return pd.DataFrame()

    for elem in contenu.iter():
        tag = elem.tag.split("}")[-1]
        code_grammaire = elem.attrib.get("code_grammaire", "")
        code_style = elem.attrib.get("code_style", "")

        if code_grammaire == "TITRE_TEXTE_DISCUSSION" and code_style == "Titre":
            texte = elem.find("ns:texte", ns)
            titre_general = texte.text.strip() if texte is not None and texte.text else ""

        elif code_grammaire == "SOUS_TITRE_TEXTE_DISCUSSION" and code_style == "Sous-tit_info":
            texte = elem.find("ns:texte", ns)
            sous_titre = texte.text.strip() if texte is not None and texte.text else ""

        elif tag == "paragraphe":
            texte_elem = elem.find("ns:texte", ns)
            if texte_elem is None:
                continue

            texte = "".join(texte_elem.itertext()).strip()
            stime = texte_elem.attrib.get("stime", "")

            orateur_elem = elem.find(".//ns:orateur", ns)
            nom, qualite, id_orateur = "", "", ""
            if orateur_elem is not None:
                nom = orateur_elem.find("ns:nom", ns)
                qualite = orateur_elem.find("ns:qualite", ns)
                id_elem = orateur_elem.find("ns:id", ns)

                nom = nom.text.strip() if nom is not None and nom.text else ""
                qualite = qualite.text.strip() if qualite is not None and qualite.text else ""
                id_orateur = id_elem.text.strip() if id_elem is not None and id_elem.text else ""

            rows.append({
                **metadata,
                "Titre_general": titre_general,
                "Sous_titre": sous_titre,
                "ID_paragraphe": elem.attrib.get("id_syceron", ""),
                "Ordre_seance": elem.attrib.get("ordre_absolu_seance", ""),
                "Code_grammaire": code_grammaire,
                "Code_style": code_style,
                "Code_parole": elem.attrib.get("code_parole", ""),
                "Role_debat": elem.attrib.get("roledebat", ""),
                "Nom_orateur": nom,
                "Qualite_orateur": qualite,
                "ID_orateur": id_orateur,
                "stime": stime,
                "Texte": texte,
                "Fichier_source": os.path.basename(fichier_xml),
            })

    return pd.DataFrame(rows)


In [2]:
def traiter_dossier_v3(dossier_path: str, pattern="*.xml", output_csv="export_v3.csv"):
    fichiers = glob.glob(os.path.join(dossier_path, pattern))
    if not fichiers:
        print("❌ Aucun fichier XML trouvé.")
        return

    print(f"🔍 Traitement de {len(fichiers)} fichiers XML dans le dossier...")

    dfs = []
    for fichier in fichiers:
        print(f"➡️  {os.path.basename(fichier)}")
        df = extraire_donnees_v3(fichier)
        if not df.empty:
            dfs.append(df)

    if dfs:
        df_concat = pd.concat(dfs, ignore_index=True)
        df_concat.to_csv(output_csv, index=False, quoting=csv.QUOTE_ALL, encoding="utf-8")
        print(f"\n✅ Export CSV : {output_csv} ({df_concat.shape[0]} lignes)")
    else:
        print("⚠️ Aucun paragraphe extrait.")


In [3]:
traiter_dossier_v3("../data/16-xml/compteRendu/")

🔍 Traitement de 605 fichiers XML dans le dossier...
➡️  CRSANR5L16S2023O1N173.xml
➡️  CRSANR5L16S2023O1N167.xml
➡️  CRSANR5L16S2023O1N198.xml
➡️  CRSANR5L16S2023E1N010.xml
➡️  CRSANR5L16S2024O1N117.xml
➡️  CRSANR5L16S2024O1N103.xml
➡️  CRSANR5L16S2023E1N004.xml
➡️  CRSANR5L16S2023O1N239.xml
➡️  CRSANR5L16S2023O1N205.xml
➡️  CRSANR5L16S2023O1N211.xml
➡️  CRSANR5L16S2024O1N088.xml
➡️  CRSANR5L16S2023O1N007.xml
➡️  CRSANR5L16S2023O1N013.xml
➡️  CRSANR5L16S2024O1N063.xml
➡️  CRSANR5L16S2024O1N077.xml
➡️  CRSANR5L16S2024O1N076.xml
➡️  CRSANR5L16S2024O1N062.xml
➡️  CRSANR5L16S2023O1N012.xml
➡️  CRSANR5L16S2023O1N006.xml
➡️  CRSANR5L16S2024O1N089.xml
➡️  CRSANR5L16S2023O1N210.xml
➡️  CRSANR5L16S2023O1N204.xml
➡️  CRSANR5L16S2023O1N238.xml
➡️  CRSANR5L16S2023E1N005.xml
➡️  CRSANR5L16S2024O1N102.xml
➡️  CRSANR5L16S2024O1N116.xml
➡️  CRSANR5L16S2023E1N011.xml
➡️  CRSANR5L16S2023O1N199.xml
➡️  CRSANR5L16S2023O1N166.xml
➡️  CRSANR5L16S2023O1N172.xml
➡️  CRSANR5L16S2023O1N158.xml
➡️  CRSANR5L16S202